In [1]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# MatrixVectorProductComp

`MatrixVectorProductComp` performs a matrix-vector product.  It may be vectorized to provide the result at one or more points simultaneously.

$$
    \bar{b}_i = \left[ A_i \right] \bar{x}_i
$$

## MatrixVectorProductComp Options

The default `vec_size` is 1, providing the matrix vector product of $a$ and $x$ at a single
point.

Other options for MatrixVectorProductComp allow the user to rename the input variables $a$ and $x$ and the output $b$, as well as specifying their units.

In [2]:
import openmdao.api as om
om.show_options_table("openmdao.components.matrix_vector_product_comp.MatrixVectorProductComp")

Option,Default,Acceptable Values,Acceptable Types,Description
A_name,A,N/A,['str'],The variable name for the matrix.
A_shape,"(3, 3)",N/A,['tuple'],The shape of the input matrix at a single point.
A_units,N/A,N/A,['str'],The units of the input matrix.
always_opt,False,"[True, False]",['bool'],"If True, force nonlinear operations on this component to be included in the optimization loop even if this component is not relevant to the design variables and responses."
b_name,b,N/A,['str'],The variable name of the output vector.
b_units,N/A,N/A,['str'],The units of the output vector.
distributed,False,"[True, False]",['bool'],"If True, set all variables in this component as distributed across multiple processes"
run_root_only,False,"[True, False]",['bool'],"If True, call compute, compute_partials, linearize, apply_linear, apply_nonlinear, and compute_jacvec_product only on rank 0 and broadcast the results to the other ranks."
vec_size,1,N/A,['int'],The number of points at which the matrix vector product is to be computed
x_name,x,N/A,['str'],The name of the input vector.


## MatrixVectorProductComp Constructor

The call signature for the `MatrixVectorProductComp` constructor is:

```{eval-rst}
    .. automethod:: openmdao.components.matrix_vector_product_comp.MatrixVectorProductComp.__init__
        :noindex:
```

## MatrixVectorProductComp Usage

There are often situations when numerous products need to be computed, essentially in parallel.
You can reduce the number of components required by having one `MatrixVectorProductComp` perform multiple operations.
This is also convenient when the different operations have common inputs.

The ``add_product`` method is used to create additional products after instantiation.

```{eval-rst}
    .. automethod:: openmdao.components.matrix_vector_product_comp.MatrixVectorProductComp.add_product
       :noindex:
```

## MatrixVectorProductComp Example

The following code demonstrates the use of the MatrixVectorProductComp, finding the product
of a random 3x3 matrix `Mat` and a 3-vector `x` at 100 points simultaneously.

In [3]:
import numpy as np
import openmdao.api as om

nn = 2

p = om.Problem()

p.model.add_subsystem(name='mat_vec_product_comp',
                      subsys=om.MatrixVectorProductComp(A_name='Mat', vec_size=nn,
                                                        b_name='y', b_units='m',
                                                        x_units='m'),
                      promotes_inputs=['Mat', 'x'])

p.setup()

p.set_val('Mat', np.random.rand(nn, 3, 3))
p.set_val('x', np.random.rand(nn, 3))

p.run_model()

print(p.get_val('mat_vec_product_comp.y', units='ft')[0, :])

[1.52386174 0.92803649 2.80768747]


In [4]:
print(p.get_val('mat_vec_product_comp.y', units='ft')[1, :])

[3.64295354 3.4583708  1.20022204]


In [5]:
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(p.get_val('mat_vec_product_comp.y', units='ft')[0, :],
                  np.dot(p['Mat'][0, :, :], p['x'][0, :]) * 3.2808399,
                  tolerance=1.0E-8)

assert_near_equal(p.get_val('mat_vec_product_comp.y', units='ft')[1, :],
                  np.dot(p['Mat'][1, :, :], p['x'][1, :]) * 3.2808399,
                  tolerance=1.0E-8)

1.5200001500014537e-09

## MatrixVectorProductComp Example with Multiple Products

When defining multiple products:

- An input name in one call to `add_product` may not be an output name in another call, and vice-versa.
- The units and shape of variables used across multiple products must be the same in each one.

In [6]:
import numpy as np

nn = 2

p = om.Problem()

mvp = om.MatrixVectorProductComp(A_name='Mat', vec_size=nn,
                                 b_name='y', b_units='m',
                                 x_units='m')

mvp.add_product(A_name='Mat', vec_size=nn,
                b_name='z', b_units='m',
                x_name='w', x_units='m')

p.model.add_subsystem(name='mat_vec_product_comp',
                      subsys=mvp,
                      promotes_inputs=['Mat', 'x', 'w'])

p.setup()

p.set_val('Mat', np.random.rand(nn, 3, 3))
p.set_val('x', np.random.rand(nn, 3))
p.set_val('w', np.random.rand(nn, 3))

p.run_model()

print(p.get_val('mat_vec_product_comp.y', units='ft')[0, :])

[0.29177684 1.98151558 1.45415561]


In [7]:
print(p.get_val('mat_vec_product_comp.y', units='ft')[1, :])

[2.60592536 2.51449764 0.93339215]


In [8]:
print(p.get_val('mat_vec_product_comp.z', units='ft')[0, :])

[2.00293056 2.1316477  1.59156689]


In [9]:
print(p.get_val('mat_vec_product_comp.z', units='ft')[1, :])

[2.01502269 1.86841606 0.59724104]


In [10]:
assert_near_equal(p.get_val('mat_vec_product_comp.y', units='ft')[0, :],
                  np.dot(p['Mat'][0, :, :], p['x'][0, :]) * 3.2808399,
                  tolerance=1.0E-8)

assert_near_equal(p.get_val('mat_vec_product_comp.y', units='ft')[1, :],
                  np.dot(p['Mat'][1, :, :], p['x'][1, :]) * 3.2808399,
                  tolerance=1.0E-8)

assert_near_equal(p.get_val('mat_vec_product_comp.z', units='ft')[0, :],
                  np.dot(p['Mat'][0, :, :], p['w'][0, :]) * 3.2808399,
                  tolerance=1.0E-8)

assert_near_equal(p.get_val('mat_vec_product_comp.z', units='ft')[1, :],
                  np.dot(p['Mat'][1, :, :], p['w'][1, :]) * 3.2808399,
                  tolerance=1.0E-8)

1.5200001718093108e-09